In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Disclaimer
After taking Machine Learning course on Kaggle, I am applying the knowledge I have lenared.
Code blocks for One Hot Encoding are taken from https://www.kaggle.com/code/alexisbcook/categorical-variables

Any feedbacks or comments are welcome... :) 

In [ ]:
data=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
data.head()

In [ ]:
target=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
print(target.shape)
target.head()

In [ ]:
data["Transported"] = data["Transported"].astype(int)

In [ ]:
print(data.shape)
print(data.isnull().sum())

In [ ]:
obj_col_name= data.select_dtypes("object").columns
print(obj_col_name)

In [ ]:
obj_col_name=obj_col_name.drop('Name')
obj_col=data[obj_col_name]

In [ ]:
obj_cols_with_missing = [col for col in obj_col if data[col].isnull().any()]

In [ ]:
obj_cols_with_missing


In [ ]:
print(data[obj_cols_with_missing].isnull().sum())

In [ ]:
print(target[obj_cols_with_missing].isnull().sum())

Missing categorical data will be replaced with 'unknown' so that those columns can be put into one hot coder

In [ ]:
data_test=data[obj_cols_with_missing].fillna('unknown')
target_test=target[obj_cols_with_missing].fillna('unknown')


In [ ]:
print(data_test.shape)
print(target_test.shape)

In [ ]:
data=data.drop(obj_cols_with_missing,axis=1)
target=target.drop(obj_cols_with_missing,axis=1)

In [ ]:
data=pd.concat([data,data_test],axis=1)
target=pd.concat([target,target_test],axis=1)

In [ ]:
print(data.shape)
print(target.shape)

In [ ]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in obj_cols_with_missing if data_test[col].nunique() < 10]
print(low_cardinality_cols)

In [ ]:
high_cardinality_cols = list(set(obj_cols_with_missing)-set(low_cardinality_cols))
print(high_cardinality_cols)

> Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard



Data in the column Cabin will be split. 

In [ ]:
test= data['Cabin'].str.split('/',n=2,expand=True)
test=test.fillna('unknown')
data['c1']=test[0]
data['c2']=test[1]
data['c3']=test[2]


In [ ]:
t_test= target['Cabin'].str.split('/',n=2,expand=True)
t_test=t_test.fillna('unknown')
target['c1']=t_test[0]
target['c2']=t_test[1]
target['c3']=t_test[2]

In [ ]:
data['VIP']=data["VIP"].replace('unknown','False')
data['CryoSleep']=data['CryoSleep'].replace('unknown','False')

data["VIP"] = data["VIP"].astype(bool)
data["CryoSleep"] = data["CryoSleep"].astype(bool)

data["VIP"] = data["VIP"].astype(int)
data["CryoSleep"] = data["CryoSleep"].astype(int)

In [ ]:
target['VIP']=target["VIP"].replace('unknown','False')
target['CryoSleep']=target['CryoSleep'].replace('unknown','False')

target["VIP"] = target["VIP"].astype(bool)
target["CryoSleep"] = target["CryoSleep"].astype(bool)

target["VIP"] = target["VIP"].astype(int)
target["CryoSleep"] = target["CryoSleep"].astype(int)

In [ ]:
cabin_cardinality_cols = [col for col in ['c1','c2','c3'] if data[col].nunique() < 10]


c1 and c3 columns are low cardinality too, so these two columns will be add to low_cardinality_cols for one hot encoding

In [ ]:
low_cardinality_cols=cabin_cardinality_cols+low_cardinality_cols
print(low_cardinality_cols)

Checking the data again

In [ ]:
data.head()

OneHotEncoding will be applied on **c1**', '**c3**', '**HomePlanet**', and '**Destination**'

In [ ]:
from sklearn.preprocessing import OneHotEncoder


# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(data[['c1','c3','HomePlanet','Destination']]))
OH_cols_target = pd.DataFrame(OH_encoder.fit_transform(target[['c1','c3','HomePlanet','Destination']]))

# One-hot encoding removed index; put it back
OH_cols_train.index = data.index
OH_cols_target.index =target.index





In [ ]:
OH_X_train = pd.concat([data, OH_cols_train], axis=1)
OH_X_test = pd.concat([target, OH_cols_target], axis=1)

In [ ]:
OH_X_train=OH_X_train.drop(['c1','c3','HomePlanet','Destination'],axis=1)
OH_X_test=OH_X_test.drop(['c1','c3','HomePlanet','Destination'],axis=1)

In [ ]:
print(OH_X_train.shape)
print(OH_X_test.shape)
OH_X_train.head()

In [ ]:
oh_obj_col_name= OH_X_train.select_dtypes("object").columns
print(oh_obj_col_name)

In [ ]:
OH_X_train['c2']=OH_X_train["c2"].replace('unknown','0')
OH_X_train['c2']=OH_X_train['c2'].astype(int)

OH_X_test['c2']=OH_X_test["c2"].replace('unknown','0')
OH_X_test['c2']=OH_X_test['c2'].astype(int)

Featured engineering done for both train and test. Now it's ready for model training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor


# Separate target from predictors
y = OH_X_train.Transported
X = OH_X_train.drop(['Transported','Name','Cabin','PassengerId'], axis=1)

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
#my_model = XGBRegressor(n_estimators=500)
#my_model.fit(X_train, y_train, 
#             early_stopping_rounds=5, 
#             eval_set=[(X_valid, y_valid)],
#             verbose=False)

In [ ]:
#predictions = my_model.predict(X_valid)
#print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

In [ ]:
#predictions

0.2687865225471023

Submitting csv file

In [ ]:
OH_X_test=OH_X_test.drop(['Name','Cabin','PassengerId'], axis=1)
#final_test=my_model.predict(OH_X_test)


In [ ]:

#output = pd.DataFrame({'PassengerId': target.PassengerId,
   #                    'Transported': final_test})
#output.to_csv('submission.csv', index=False)

The following code block was taken from https://www.kaggle.com/code/eisgandar/spaceship-titanic-competition-prediction 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

xgbc = XGBClassifier()

xgbc_params = {
        "gamma": [0.5, 1, 1.5],
        "subsample": [0.6, 0.8, 1.0],
        "colsample_bytree": [0.6, 0.8, 1.0],
        "max_depth": [3, 4, 5],
        "n_estimators": [80, 100, 150]
}

xgbc_cv_model = GridSearchCV(xgbc, xgbc_params, cv = 10, n_jobs = -1)
xgbc_cv_model.fit(X_train, y_train)

In [ ]:
xgbc_tuned = XGBClassifier(gamma = xgbc_cv_model.best_params_["gamma"],
                           subsample = xgbc_cv_model.best_params_["subsample"],
                           max_depth = xgbc_cv_model.best_params_["max_depth"],
                           colsample_bytree = xgbc_cv_model.best_params_["colsample_bytree"],
                           n_estimators = xgbc_cv_model.best_params_["n_estimators"])
xgbc_tuned = xgbc_tuned.fit(X_train, y_train)

In [ ]:
y_pred = xgbc_tuned.predict(X_valid)
accuracy_score(y_valid, y_pred)

Final prediction before submission

In [ ]:
final_y_pred=xgbc_tuned.predict(OH_X_test)

In [ ]:
submission = pd.DataFrame({"PassengerId": target.PassengerId, "Transported": final_y_pred})
submission.head()

*Found out the my score was 0, I realized I need to convert Transported Column into boolean*

In [ ]:
submission['Transported']=submission['Transported'].astype(bool)

In [ ]:
submission.to_csv('submission.csv', index = False)